In [4]:
import langchain
print(langchain.__version__
)

1.1.1


In [13]:
!nvidia-smi

Fri Dec  5 11:14:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.07                 Driver Version: 566.07         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   40C    P8              4W /   35W |    1314MiB /   8188MiB |     13%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os, sys
from dotenv import load_dotenv

from telethon import TelegramClient, errors
import asyncio
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
import torch
print(f"PyTorch версия: {torch.__version__}")
print(f"Версия CUDA в PyTorch: {torch.version.cuda}")
print("Доступность cuda", torch.cuda.is_available())

nltk.download("stopwords")
sw = stopwords.words("russian")

load_dotenv()

api_id = os.getenv('APP_ID_TG')
api_hash = os.getenv('API_HASH_TG')
mistral_api_key = os.getenv('MISTRAL_API_KEY')
channels = [
    "@tass_agency",
    "@rian_ru",
    "@kommersant",
    "@gazeta_ru",
    "@meduzalive",
    "@rbc_news"
]

PyTorch версия: 2.9.1+cu126
Версия CUDA в PyTorch: 12.6
Доступность cuda True


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1 Подготовка данных

In [6]:
def clean_text_ru(text): 
   # Приведение текста к нижнему регистру
    text = text.lower()
    # Замена всех не-словесных символов на пробел (кроме букв и знаков препинания)
    text = re.sub(r'\W+', ' ', text)

    # Удаление URL-адресов
    text = re.sub(r"http\S+", "", text)

    # Создание шаблона для HTML-тегов
    html = re.compile(r'&lt;.*?&gt;')

    # Удаление HTML-тегов из текста
    text = html.sub(r'', text)

    # Список пунктуаций для удаления
    punctuations = '@#!?+&amp;*[]-%.:/();$=&gt;&lt;|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p, '')  # Удаление пунктуации

    # Удаление стоп-слов и приведение слов к нижнему регистру
    text = [word.lower() for word in text.split() if word.lower() not in sw]

    # Объединение слов обратно в текст
    text = " ".join(text)

    # Создание шаблона для поиска эмодзи
    emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # эмоции
                        u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                        u"\U0001F680-\U0001F6FF"  # транспорт и карты
                        u"\U0001F1E0-\U0001F1FF"  # флаги
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)

    # Удаление эмодзи из текста
    text = emoji_pattern.sub(r'', text)

    return text

In [7]:
client = TelegramClient("session_news", api_id, api_hash)


async def load_telegram_news(channel, limit=100):
    """
    Загружает последние сообщения из указанного Telegram-канала.
    """
    try:
        print(f"Подключение к Telegram для обработки канала {channel}...")
        await client.start()

        docs = []
        print(f"Загрузка сообщений из канала {channel}...")
        async for msg in client.iter_messages(channel, limit=limit):
            if not msg.text:
                continue

            # Очистка текста
            cleaned_text = clean_text_ru(msg.text)
            # print(f"Оригинальный текст: {msg.text[:50]}...")
            # print(f"Очищенный текст: {cleaned_text[:50]}...")

            docs.append({
                "id": msg.id,
                "channel": channel,
                "text": cleaned_text,
                "date": msg.date.isoformat()
            })
        print(f"Загрузка из канала {channel} завершена. Получено {len(docs)} сообщений.")
        return docs
    except errors.UsernameInvalidError:
        print(f"Канал {channel} недоступен или не существует.")
        return []
    except errors.FloodWaitError as e:
        print(f"Превышен лимит запросов для канала {channel}. Ожидание {e.seconds} секунд.")
        await asyncio.sleep(e.seconds)
        return []
    except Exception as e:
        print(f"Ошибка при обработке канала {channel}: {e}")
        return []
    finally:
        print(f"Отключение от Telegram после обработки канала {channel}...")
        await client.disconnect()


def prepare_rag_documents(messages):
    """
    Преобразует сообщения в формат, подходящий для RAG.
    """
    print("Подготовка данных для RAG...")
    print(f"Количество сообщений для подготовки: {len(messages)}")  # Отладка
    rag_docs = [
        {
            "id": msg["id"],
            "content": msg["text"],
            "metadata": {
                "source": "telegram",
                "channel": msg["channel"],
                "date": msg["date"]
            }
        }
        for msg in messages
    ]
    print(f"Подготовлено {len(rag_docs)} документов.")
    return rag_docs


async def main():
    all_messages = []
    for channel in channels:
        print(f"Обработка канала {channel}...")
        messages = await load_telegram_news(channel, limit=100)
        print(f"Сообщений из канала {channel}: {len(messages)}")  # Отладка
        all_messages.extend(messages)

    if all_messages:
        print("Подготовка новостей для RAG...")
        rag_docs = prepare_rag_documents(all_messages)
        print("Пример подготовленного документа:")
        print(rag_docs[0] if rag_docs else "Нет документов для отображения.")  # Отладка
        return all_messages, rag_docs
    else:
        print("Нет доступных сообщений.")
        return [], []


# Запуск main() и получение данных
all_messages, rag_docs = await main()
print(f"Всего сообщений: {len(all_messages)}")  # Отладка
print(f"Всего документов для RAG: {len(rag_docs)}")  # Отладка

Обработка канала @tass_agency...
Подключение к Telegram для обработки канала @tass_agency...
Загрузка сообщений из канала @tass_agency...
Загрузка сообщений из канала @tass_agency...
Загрузка из канала @tass_agency завершена. Получено 89 сообщений.
Отключение от Telegram после обработки канала @tass_agency...
Сообщений из канала @tass_agency: 89
Обработка канала @rian_ru...
Подключение к Telegram для обработки канала @rian_ru...
Загрузка из канала @tass_agency завершена. Получено 89 сообщений.
Отключение от Telegram после обработки канала @tass_agency...
Сообщений из канала @tass_agency: 89
Обработка канала @rian_ru...
Подключение к Telegram для обработки канала @rian_ru...
Загрузка сообщений из канала @rian_ru...
Загрузка сообщений из канала @rian_ru...
Загрузка из канала @rian_ru завершена. Получено 91 сообщений.
Отключение от Telegram после обработки канала @rian_ru...
Сообщений из канала @rian_ru: 91
Обработка канала @kommersant...
Подключение к Telegram для обработки канала @komme

In [8]:
rag_docs[:3]

[{'id': 351656,
  'content': 'президентским дворцом индии прошла церемония официальной встречи путина видео ирина полина тасс',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-05T06:03:10+00:00'}},
 {'id': 351655,
  'content': 'задержаны 11 подозреваемых хищении 5 2 млн рублей военных убывших зону сво сообщила ирина волк отмечается злоумышленники искали информацию гражданах которые хотят заключить контракт мо рф сообщники убеждали людей передать банковские карты либо оформить доступ счету помощью доверенности видео официальный eer канал представителя мвд россии ирины волк тасс',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-05T05:52:01+00:00'}},
 {'id': 351654,
  'content': 'главное заявлений юрия ушакова телефонного разговора трампом графике путина пока необходимости легко организуется заявил ответ вопрос тасс кремль ждет реакцию американских коллег итогам обсуждения состоявшегося вторник путиным уиткоффом

### 2 Подготовка поисквого индекса

In [14]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=[
        "\n\n",
        "\n",
        ". ",
        "! ",
        "? ",
        ", ",
        " ",
        ""
    ]
)

def to_langchain_docs(raw_list):
    docs = []
    for item in raw_list:
        docs.append(
            Document(
                page_content=item["content"],
                metadata=item["metadata"]
            )
        )
    return docs

def split_docs(rag_docs):
    chunks = splitter.split_documents(rag_docs)
    return chunks

In [15]:
docs = to_langchain_docs(rag_docs)
chunks = split_docs(docs)

In [16]:
chunks[:3]

[Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-05T06:03:10+00:00'}, page_content='президентским дворцом индии прошла церемония официальной встречи путина видео ирина полина тасс'),
 Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-05T05:52:01+00:00'}, page_content='задержаны 11 подозреваемых хищении 5 2 млн рублей военных убывших зону сво сообщила ирина волк отмечается злоумышленники искали информацию гражданах которые хотят заключить контракт мо рф сообщники убеждали людей передать банковские карты либо оформить доступ счету помощью доверенности видео официальный eer канал представителя мвд россии ирины волк тасс'),
 Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-05T05:42:24+00:00'}, page_content='главное заявлений юрия ушакова телефонного разговора трампом графике путина пока необходимости легко организуется заявил ответ вопрос тасс кремль ждет реакцию американских коллег 

### 3 Embedder

In [19]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


### 4 Подготовка поискового движка

In [25]:
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

### 5 Построение Retrieval-Augmented Generation (RAG)

In [27]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-small-latest",
    max_retries=2,
    api_key=mistral_api_key
)
llm.verbose = False

In [39]:
llm.invoke('Hi!')

AIMessage(content='Hello! 😊 How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 5, 'total_tokens': 18, 'completion_tokens': 13}, 'model_name': 'mistral-small-latest', 'model': 'mistral-small-latest', 'finish_reason': 'stop'}, id='lc_run--5bd16e2c-8781-4654-bc62-15bb9771dbdd-0', usage_metadata={'input_tokens': 5, 'output_tokens': 13, 'total_tokens': 18})

In [37]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.prompts import PromptTemplate

# Создаем ретривер на основе векторного хранилища
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Создаем PromptTemplate для вставки документов
prompt = PromptTemplate(
    input_variables=["context", "input"],
    template="Контекст: {context}\n\nВопрос: {input}\n\nОтвет:"
)

# 1) Chain, который вставляет документы в prompt
document_chain = create_stuff_documents_chain(llm, prompt)
# 2) Chain "retriever + LLM"
rag_chain = create_retrieval_chain(retriever, document_chain)

# 3) Использование
response = rag_chain.invoke({"input": "Есть ли новости, влияющие на курс акций нефтянных компаний?"})
print(response["answer"])

Да, в предоставленном контексте есть несколько новостей, которые могут повлиять на курс акций нефтяных компаний:

1. **Санкции против NIS (сербской нефтяной компании)** – несмотря на то, что заправки Лукойла в Сербии продолжат работать благодаря генеральной лицензии Минфина США, это может создать неопределенность в отношении других активов нефтяных компаний, особенно в странах, где действуют санкции.

2. **Продление лицензии Минфина США для Лукойла** – продление разрешения на операции с зарубежными автозаправками Лукойла до 29 апреля 2026 года снижает риски для компании, но также может указывать на возможные ограничения в будущем.

3. **Динамика курса рубля** – ослабление рубля (курс доллара на форекс упал ниже 76) может повлиять на финансовые показатели нефтяных компаний, работающих в России, так как их доходы в рублях могут снижаться в пересчете на доллары.

4. **Общая нестабильность валютного рынка** – хрупкость баланса спроса и предложения на валютном рынке может усилить волатильно